# 🪐 Interplanetary Travel
## How to Get from Earth to Mars (and Beyond!)

### What You'll Learn
1. Hohmann transfer orbits
2. Delta-V requirements for different destinations
3. Launch windows and planetary alignment
4. Why timing is everything in space travel

---

## 1. The Challenge of Interplanetary Travel

Getting to another planet isn't just about going in a straight line. In space, everything is **orbiting** - Earth orbits the Sun, Mars orbits the Sun, and your spacecraft will orbit the Sun too!

The key insight: **Space travel is about changing your orbit to intersect with your destination's orbit.**

### Why Can't We Just Go Straight?
- A "straight line" to Mars would require ENORMOUS amounts of fuel
- We'd have to fight the Sun's gravity the entire way
- Instead, we use the Sun's gravity to our advantage


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Solar system constants
AU = 1.496e11  # Astronomical Unit in meters
MU_SUN = 1.327e20  # Sun's gravitational parameter (m³/s²)

# Planet orbital radii (in AU)
PLANETS = {
    'Mercury': 0.387,
    'Venus': 0.723,
    'Earth': 1.000,
    'Mars': 1.524,
    'Jupiter': 5.203,
    'Saturn': 9.537,
}

print("✅ Ready to plan interplanetary missions!")


## 2. The Hohmann Transfer: The Most Efficient Route

A **Hohmann transfer** is an elliptical orbit that touches both the starting orbit and the destination orbit. It's the most fuel-efficient way to travel between two circular orbits.

### How It Works
1. **Burn 1 (Departure):** At Earth, fire engines to speed up and enter the transfer ellipse
2. **Coast:** Travel along the ellipse with NO engine burns (just falling!)
3. **Burn 2 (Arrival):** At Mars, fire engines to slow down and match Mars' orbit

### The Math

**Transfer orbit semi-major axis:**
$$a_{transfer} = \frac{r_1 + r_2}{2}$$

**Transfer time (half the orbital period):**
$$T_{transfer} = \pi \sqrt{\frac{a^3}{\mu_{Sun}}}$$


In [ ]:
def hohmann_transfer(r1_au, r2_au):
    """
    Calculate Hohmann transfer parameters between two circular orbits.
    
    Parameters:
    -----------
    r1_au : float - Departure orbit radius (AU)
    r2_au : float - Arrival orbit radius (AU)
    
    Returns: dict with transfer parameters
    """
    r1 = r1_au * AU  # Convert to meters
    r2 = r2_au * AU
    
    # Transfer ellipse semi-major axis
    a_transfer = (r1 + r2) / 2
    
    # Transfer time (half period)
    T_transfer = np.pi * np.sqrt(a_transfer**3 / MU_SUN)
    
    # Velocities
    v1_circular = np.sqrt(MU_SUN / r1)  # Initial circular orbit
    v2_circular = np.sqrt(MU_SUN / r2)  # Final circular orbit
    
    # Transfer orbit velocities at perihelion and aphelion
    v_perihelion = np.sqrt(MU_SUN * (2/r1 - 1/a_transfer))
    v_aphelion = np.sqrt(MU_SUN * (2/r2 - 1/a_transfer))
    
    # Delta-V for each burn
    dv1 = abs(v_perihelion - v1_circular)
    dv2 = abs(v2_circular - v_aphelion)
    
    return {
        'semi_major_axis_au': a_transfer / AU,
        'transfer_time_days': T_transfer / 86400,
        'dv1_km_s': dv1 / 1000,
        'dv2_km_s': dv2 / 1000,
        'total_dv_km_s': (dv1 + dv2) / 1000
    }

# Calculate Earth to Mars transfer
mars_transfer = hohmann_transfer(1.0, 1.524)

print("🚀 EARTH TO MARS HOHMANN TRANSFER")
print("=" * 50)
print(f"Transfer orbit size: {mars_transfer['semi_major_axis_au']:.3f} AU")
print(f"Transit time: {mars_transfer['transfer_time_days']:.0f} days ({mars_transfer['transfer_time_days']/30:.1f} months)")
print(f"\nDelta-V Budget:")
print(f"  Departure burn: {mars_transfer['dv1_km_s']:.2f} km/s")
print(f"  Arrival burn:   {mars_transfer['dv2_km_s']:.2f} km/s")
print(f"  TOTAL:          {mars_transfer['total_dv_km_s']:.2f} km/s")


## 3. Compare Missions to Different Planets

Let's see how delta-V and travel time change for different destinations:


In [ ]:
# Calculate transfers to all planets
print("🪐 HOHMANN TRANSFERS FROM EARTH")
print("=" * 70)
print(f"{'Destination':<12} {'Distance':>10} {'Transit Time':>15} {'Total ΔV':>12}")
print("-" * 70)

destinations = ['Venus', 'Mars', 'Jupiter', 'Saturn']
results = []

for planet in destinations:
    transfer = hohmann_transfer(1.0, PLANETS[planet])
    days = transfer['transfer_time_days']
    
    if days < 365:
        time_str = f"{days:.0f} days"
    else:
        time_str = f"{days/365:.1f} years"
    
    print(f"{planet:<12} {PLANETS[planet]:>8.2f} AU {time_str:>15} {transfer['total_dv_km_s']:>10.2f} km/s")
    results.append((planet, transfer['total_dv_km_s'], days))

print("-" * 70)
print("\n💡 Notice: Outer planets need less delta-V but MUCH more time!")
print("   Jupiter takes ~3 years, Saturn takes ~6 years via Hohmann transfer.")


## 4. Launch Windows: Why Timing Matters

You can't just launch to Mars whenever you want! The planets must be **aligned correctly**.

### Phase Angle
Mars must be at a specific angle ahead of Earth when you launch. This angle ensures Mars will be at the right place when your spacecraft arrives ~9 months later.

**Phase angle formula:**
$$\phi = 180° - \omega_{Mars} \times T_{transfer}$$

### Synodic Period
Launch windows to Mars open approximately every **26 months** (2.14 years).

This is the time it takes for Earth and Mars to return to the same relative positions.


In [ ]:
# Calculate launch windows
T_EARTH = 365.25  # days
T_MARS = 687.0    # days

# Synodic period formula: 1/T_syn = |1/T1 - 1/T2|
synodic_period = 1 / abs(1/T_EARTH - 1/T_MARS)

# Phase angle for Mars
transfer_time = mars_transfer['transfer_time_days']
omega_mars = 360 / T_MARS  # degrees per day
phase_angle = 180 - omega_mars * transfer_time

print("📅 MARS LAUNCH WINDOWS")
print("=" * 50)
print(f"Transfer time: {transfer_time:.0f} days")
print(f"Phase angle required: {phase_angle:.1f}°")
print(f"Synodic period: {synodic_period:.0f} days ({synodic_period/365:.2f} years)")
print(f"\n⏰ Launch windows occur every ~{synodic_period/30:.0f} months!")
print("\nRecent Mars launch windows:")
years = [2018, 2020, 2022, 2024, 2026]
for year in years:
    print(f"  • {year}")


## 🎓 Key Takeaways

1. **Hohmann transfers are the most efficient** - Use an ellipse touching both orbits

2. **Earth to Mars: ~259 days, 5.6 km/s total delta-V**

3. **Launch windows are limited** - Must wait for correct planetary alignment

4. **Windows open every ~26 months** - Miss one and wait 2+ years!

5. **Outer planets take years** - Jupiter ~3 years, Saturn ~6 years

---

## 🚀 Try the Simulator!

Run the Mars Mission Simulator to see this in action:

```bash
cd ../Projects/Starship_Trajectory_Planner
python mission_planner.py
```

---

*"Mars is there, waiting to be reached."* — Buzz Aldrin
